# Source detection and fit

In [1]:
#useful to make plots editable in the notebook
%matplotlib notebook 

In [2]:
#We prefer our implementation of the SpectralCube
from sitelle.process import SpectralCubePatch as SpectralCube
from sitelle.constants import FITS_DIR


/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
